In [12]:
import argparse
import os
import pandas as pd
import yaml
from tqdm import tqdm
from datetime import datetime
config = yaml.load(open('../../config/config.yaml'))


In [6]:
args = {
    'days': 60,
    'save_dir': 'images/'
}

In [13]:
nc_down = pd.read_csv('../../'+config['data']['raw']['train']['nc_down']) 
nc_exception_train = pd.read_csv('../../'+config['data']['process']['train']['nc_exceptions_key_ip_column_name_value_timelist'])
nc_exception_test = pd.read_csv('../../'+config['data']['process']['test']['nc_exceptions_key_ip_column_name_value_timelist'])

In [15]:
# nc_down.join(nc_exception_test.set_index('nc_ip'), on='nc_ip', how='inner')
nc_down_with_exceptions = nc_down.join(nc_exception_train.set_index('nc_ip'), on='nc_ip', how='left')

In [16]:
exception_name_columns = config['exception_name']['train']
for exception_name in exception_name_columns:
    nc_down_with_exceptions[exception_name] = nc_down_with_exceptions[exception_name].fillna('[]').apply(eval)
    nc_exception_test[exception_name] = nc_exception_test[exception_name].fillna('[]').apply(eval)

In [22]:
# for i in tqdm(range(nc_down_with_exceptions.shape[0]), total=nc_down_with_exceptions.shape[0]):
#     nc = nc_down_with_exceptions.loc[i]
#     for exception_name in exception_name_columns:
        
    

,nc_ip,down_start_time,is_active,exception_name_10,exception_name_11,exception_name_12,exception_name_14,exception_name_15,exception_name_16,exception_name_17,...,exception_name_55,exception_name_56,exception_name_57,exception_name_58,exception_name_59,exception_name_6,exception_name_60,exception_name_7,exception_name_8,exception_name_9
0,f1e10ce9-f8f9-4114-8fb0-c83a52947844,2020-03-10 16:27:20,True,[],[],[],[],"[2020-03-05 14:04:37, 2020-03-05 14:05:43, 202...",[],[],...,[],[],"[2020-03-05 14:03:24, 2020-03-05 14:03:24, 202...","[2020-03-05 14:03:24, 2020-03-05 14:03:24, 202...",[],[],[],[],[],"[2020-03-05 14:03:24, 2020-03-05 14:03:24, 202..."
1,e3b9909a-d9a0-4188-ad30-1d131ccb60dc,2019-11-15 09:25:24,True,"[2019-11-15 08:49:54, 2019-11-15 08:50:13, 201...","[2019-11-15 08:51:19, 2019-11-15 08:52:20, 201...","[2019-11-15 08:52:46, 2019-11-15 08:52:46, 201...",[],"[2019-11-15 08:50:55, 2019-11-15 08:51:57, 201...",[],[],...,[],[],"[2019-11-15 08:49:53, 2019-11-15 08:50:13, 201...","[2019-11-15 08:49:53, 2019-11-15 08:49:53, 201...",[],[],[],[],[],"[2019-11-15 08:49:54, 2019-11-15 08:50:13, 201..."
2,b539e341-4a45-4001-9aa1-f6a44f996552,2016-12-18 12:38:12,True,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
3,9e0d0a75-5796-41e5-845b-558f3ef32685,2019-08-08 02:44:26,True,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
4,bcb19ebd-58b6-4114-b266-2cd3df8c7867,2020-01-07 13:00:40,True,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,9988b887-4fe9-48de-b207-eab2407100f8,2020-03-29 17:01:42,True,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
202,ab535065-0b03-4368-8065-569fdd9dc2e2,2018-04-19 06:04:47,False,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
203,4ec32f4e-bd98-4865-8da2-0a06fdc6c350,2020-10-02 12:10:08,True,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
204,7ec737a3-ae07-4db3-a689-90dff57010bf,2019-07-31 21:14:50,True,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]


In [7]:
# plot
# 打印到 nc_ip_dim 维度里
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
cmap = plt.cm.coolwarm
str2date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").date() if '.' in x else datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()
for k in tqdm(range(nc_down_with_exceptions.shape[0]), total=nc_down_with_exceptions.shape[0]):

    down_time = str2date(nc_down_with_exceptions.loc[k]['down_start_time'])
    max_cnt = 0
    for i, exception_name in enumerate(exception_name_columns):
        nc = nc_down_with_exceptions.loc[k][exception_name]
        if len(nc) > 0:
            list_of_datetimes = [str2date(c) for c in nc]
            items = sorted(Counter(list_of_datetimes).items())
            dates = [x for x, y in items]
            cnt = [y for x, y in items]
            dates = matplotlib.dates.date2num(dates)

            plt.plot_date(dates, cnt, ms=2, lw=1, ls='-', color=cmap(i / len(exception_name_columns)), label=exception_name)
            max_cnt = max(max_cnt, max(cnt))
    plt.vlines(down_time, ymin=0, ymax=max(cnt) + 1, colors = "grey", linestyles = "dashed")
    if max_cnt > 0:
        plt.legend(framealpha=0.5)
    plt.title = "nc_ip: " + nc_down_with_exceptions.loc[k]['nc_ip'] + "\nis_active:" + str(nc_down_with_exceptions.loc[k]['is_active'])
    plt.xlabel = "Exception Date"
    plt.ylabel = "Exception Count"
    plt.xticks(fontsize='x-small')
    plt.savefig('images/train/nc_ip_dim/{}.png'.format(nc_down_with_exceptions.loc[k]['nc_ip']), dpi=100, facecolor='white')
    plt.close()

100%|██████████| 206/206 [00:20<00:00,  9.82it/s]


In [7]:
# plot
# exception_dim 里
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
cmap = plt.cm.coolwarm
str2date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").date() if '.' in x else datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()
for k in tqdm(range(nc_down_with_exceptions.shape[0]), total=nc_down_with_exceptions.shape[0]):

    down_time = str2date(nc_down_with_exceptions.loc[k]['down_start_time'])
    max_cnt = 0
    for i, exception_name in enumerate(exception_name_columns):
        nc = nc_down_with_exceptions.loc[k][exception_name]
        if len(nc) > 0:
            list_of_datetimes = [str2date(c) for c in nc]
            items = sorted(Counter(list_of_datetimes).items())
            dates = [x for x, y in items]
            cnt = [y for x, y in items]
            dates = matplotlib.dates.date2num(dates)

            plt.plot_date(dates, cnt, ms=2, lw=1, ls='-', color=cmap(i / len(exception_name_columns)), label=exception_name)
            plt.vlines(down_time, ymin=0, ymax=max(cnt) + 1, colors = "grey", linestyles = "dashed")
            max_cnt = max(cnt)
            if max_cnt > 0:
                plt.legend(framealpha=0.5)
            plt.title = "nc_ip: " + nc_down_with_exceptions.loc[k]['nc_ip'] + "\nis_active:" + str(nc_down_with_exceptions.loc[k]['is_active'])
            plt.xlabel = "Exception Date"
            plt.ylabel = "Exception Count"
            plt.xticks(fontsize='x-small')
            if not os.path.exists('images/train/exception_dim/{}'.format(exception_name)):
                os.makedirs('images/train/exception_dim/{}'.format(exception_name))
            plt.savefig('images/train/exception_dim/{}/{}.png'.format(exception_name, nc_down_with_exceptions.loc[k]['nc_ip']), dpi=100, facecolor='white')
            plt.close()

 75%|███████▍  | 154/206 [00:36<00:19,  2.64it/s]

In [18]:
# 打印test
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
cmap = plt.cm.coolwarm
str2date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").date() if '.' in x else datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()
for k in tqdm(range(nc_exception_test.shape[0]), total=nc_exception_test.shape[0]):

    for i, exception_name in enumerate(exception_name_columns):
        nc = nc_exception_test.loc[k][exception_name]
        if len(nc) > 0:
            list_of_datetimes = [str2date(c) for c in nc]
            items = sorted(Counter(list_of_datetimes).items())
            dates = [x for x, y in items]
            cnt = [y for x, y in items]
            dates = matplotlib.dates.date2num(dates)

            plt.plot_date(dates, cnt, ms=2, lw=1, ls='-', color=cmap(i / len(exception_name_columns)), label=exception_name)
    plt.legend(framealpha=0.5)
    plt.xticks(fontsize='x-small')
    plt.savefig('images/test/nc_ip_dim/{}.png'.format(nc_exception_test.loc[k]['nc_ip']), dpi=100, facecolor='white')
    plt.close()

100%|██████████| 3631/3631 [08:35<00:00, 11.71it/s]
